In [ ]:
import random
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import confusion_matrix
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import acf

from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.diagnostic import kstest_normal
from statsmodels.stats.diagnostic import het_breuschpagan

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.preprocessing import StandardScaler

from math import sqrt
from math import floor
from math import isnan
from scipy import stats
from datetime import date
from datetime import datetime

from itertools import product
from itertools import groupby

!pip install arch
from arch import arch_model

from scipy import optimize
from scipy.optimize import minimize

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.filterwarnings('ignore', message="RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds")
warnings.filterwarnings('ignore', message="Values in x were outside bounds during a minimize step, clipping to bounds")

## Import Data and Compute Log Returns

In this section, we import the historical prices of IUIT.L from Yahoo Finance using the yfinance library. We then compute daily log returns by taking the natural logarithm of the closing prices and subtracting the previous day's log price. To remove outliers, we remove any returns that are more than 3 standard deviations from the mean. This is done to ensure that our analysis is not skewed by extreme events.

In [2]:
def readYfData(idx):
  start_date = datetime(2016, 1, 1)
  end_date   = date.today()
  etf_df = yf.download(idx, start=start_date, end=end_date)
  return etf_df

def computeReturns(prices):
  diffs   = np.diff(prices).tolist()
  prices  = np.delete(prices, len(prices) - 1).tolist()
  returns = [round(a / b, 7) for a,b in zip(diffs, prices)]
  returns = [0.00000] + returns
  return returns

def removeOutliers(returns, sigma_limit):
  scaler = StandardScaler()
  scaled_log_returns = scaler.fit_transform(np.array(returns).reshape(-1, 1))
  scaled_outliers = [i for i, x in enumerate(scaled_log_returns)\
                     if abs(x) > sigma_limit]
  rep = np.median(returns)
  log_returns = [rep if i in scaled_outliers else x for\
                 i, x in enumerate(returns)]
  return log_returns

In [ ]:
iuit_l = readYfData('IUIT.L')
print('Last day loaded = {}'.format(iuit_l.index[-1]))

Here we define constants that help us control the training process:

In [4]:
# Define constants to control trading
ENTRY_IDX   = 1489 # Index of 19-11-2021 when we started trading
LOOK_BACK   =  365 # Number of days to look back when building a model
LOOK_AHEAD  =    3 # Number of days to forecast ahead
BURN_IN     =   30 # Burn in period to allow RSI to stabilize
OUTLIER_SIG =  3.3 # Outlier sigma to cut-off outliers
INITIAL_INV = 1000 # Initial investment amount
TRANS_COST  =    8 # Cost per one transaction
IDLE_COST   =  120 # Cost if no transactions in 365 days
MIN_PURCHAS =   10 # Minimal number of stocks to buy in a batch
MAX_P       =    1
MAX_Q       =    1

prices      = iuit_l['Close'].tolist()
log_prices  = np.log(prices).tolist()
returns     = computeReturns(log_prices)
price_trend = pd.Series(prices).rolling(100).mean()

# We use standardized log returns to detect outliers since other methods like
# IQR are too sensitive to outliers and identify too many of them in the series
# which might lead to loosing some valuable information.
returns  = removeOutliers(returns, OUTLIER_SIG)
n_steps  = len(returns) - (ENTRY_IDX + LOOK_AHEAD)
last_idx = len(returns) + 1 - LOOK_AHEAD

# Note that the last index must be the length of history + 1 minus look ahead period
# to confirm that the split was done correctly
print('Steps = {}, history = {}, last index = {}, look ahead = {}'.format(
    n_steps, len(returns), last_idx, LOOK_AHEAD))

Steps = 549, history = 2041, last index = 2039, look ahead = 3


## Establish a Baseline Performance Benchmark

To evaluate the effectiveness of our algorithmic trading strategies, a baseline performance benchmark is established using a buy-and-hold strategy. This benchmark represents the potential profit margin achievable by simply purchasing and holding IUIT.L shares over the entire historical period. The Relative Strength Index (RSI) indicator is employed to determine buy and sell signals, ensuring that our benchmark reflects a prudent approach to stock market participation.

In [5]:
def computeRsi(returns):
  up_close   = len(list(filter(lambda x: (x > 0), returns)))
  down_close = len(list(filter(lambda x: (x < 0), returns)))
  if down_close == 0:
    down_close = 1
  rs_val = round(up_close / down_close, 2)
  rsi    = round(100 - (100 / (1 + rs_val)), 2)
  return rsi

def calculateAverageReturn(daily):
  cumul_r = 1
  for r in daily:
    cumul_r *= (1 + r)
  return cumul_r

In [6]:
## Naive trading baseline -- DO NOT DELETE, as we need it for comparison
cash   = INITIAL_INV
stocks = 0

equity    = []
inventory = []

for step in range(ENTRY_IDX, last_idx):
  today_price = prices[step]
  window_ret  = returns[step - LOOK_BACK:step]
  rsi = computeRsi(window_ret[-7:])
  signal = 'hold'
  if rsi >= 80:
    signal = 'sell'
  elif rsi <= 30:
    signal = 'buy'

  if signal == 'buy':
    affordable_qty = floor((cash - TRANS_COST) / today_price)
    if affordable_qty >= MIN_PURCHAS:
      cash   -= today_price * affordable_qty
      stocks += affordable_qty
  if signal == 'sell':
    if stocks > 0:
      cash  += today_price * stocks
      stocks = 0
  equity.append(cash)
  inventory.append(stocks)

  if step == last_idx - 1:
    today_date = iuit_l.index[step]
    print('Date = {}, price = {:.2f}, signal = {}'.format(today_date,
                                                          today_price,
                                                          signal))
    cash  += today_price * stocks
    stocks = 0

profit = 100 * (cash - INITIAL_INV) / INITIAL_INV
print('Profit: naive baseline = {:.2f}%'.format(profit))

Date = 2024-01-29 00:00:00, price = 26.13, signal = hold
Profit: naive baseline = 14.45%


## Implement the Moving Average Convergence Divergence Strategy

To enhance our trading performance beyond the buy-and-hold benchmark, we employ the Moving Average Convergence Divergence (MACD) indicator. This technical analysis tool assesses the relationship between two exponential moving averages (EMAs) to identify potential trend reversals. By analyzing the crossovers of the MACD and signal lines, we generate buy and sell signals that aim to capitalize on favorable market conditions and outperform the benchmark strategy.

In [ ]:
# MACD trading attempt to perform better than the market
# Updates the baseline that we have to beat with machine learning
ema12 = pd.Series(prices).ewm(12).mean()
ema26 = pd.Series(prices).ewm(26).mean()
macd  = ema12 - ema26
ema9  = macd.ewm(9).mean()
macd  = macd.tolist()
ema9  = ema9.tolist()

cash   = INITIAL_INV
stocks = 0

equity    = []
inventory = []
signals   = []

for step in range(ENTRY_IDX, last_idx):
  today_price = prices[step]
  macd_yesterday = macd[step - 1] - ema9[step]
  macd_today = macd[step] - ema9[step]
  signal = 'hold'
  if macd_yesterday > 0 and macd_today < 0:
    signal = 'buy'
  elif macd_yesterday < 0 and macd_today > 0:
    signal = 'sell'
  if signal == 'buy':
    affordable_qty = floor((cash - TRANS_COST) / today_price)
    if affordable_qty >= MIN_PURCHAS:
      cash -= today_price * affordable_qty
      stocks += affordable_qty
  if signal == 'sell':
    if stocks > 0:
      cash += today_price * stocks
      stocks = 0
  equity.append(cash)
  inventory.append(stocks)
  signals.append(signal)

profit = 100 * (cash - INITIAL_INV) / INITIAL_INV
print('Profit: MACD crossover = {:.2f}%'.format(profit))

Current trading example allowing to show the progress of the trading and the last day signal together with the price

In [8]:
def plotPrices(prices, signals, entry_idx, start, end):
  """
  This function plots price and the signals produced by the trading algorithm
  """
  if start == None:
    start = 0
  if end == None:
    end = len(signals)
  subset_prices = prices[ENTRY_IDX:]
  subset_prices = subset_prices[start:end]
  subset_signal = signals[start:end]
  subset_index  = range(len(subset_prices))

  sub_dt = pd.DataFrame({'price':subset_prices, 'signal':subset_signal})
  x1 = sub_dt[sub_dt['signal'] == 'buy'].index
  y1 = sub_dt[sub_dt['signal'] == 'buy']['price']
  x2 = sub_dt[sub_dt['signal'] == 'sell'].index
  y2 = sub_dt[sub_dt['signal'] == 'sell']['price']

  plt.plot(subset_index, subset_prices, color='lightgrey', zorder=0)
  plt.scatter(x1, y1, marker='o', s=6, color='red', zorder=1, label='buy')
  plt.scatter(x2, y2, marker='o', s=6, color='green', zorder=1, label='sell')
  plt.legend()
  plt.show()

In [ ]:
# MACD crossover trading until the last day
ema12 = pd.Series(prices).ewm(12).mean()
ema26 = pd.Series(prices).ewm(26).mean()
macd  = ema12 - ema26
ema9  = macd.ewm(9).mean()
macd  = macd.tolist()
ema9  = ema9.tolist()

cash   = INITIAL_INV
stocks = 0

equity    = []
inventory = []
signals   = []

for step in range(ENTRY_IDX, len(returns)):
  today_price = prices[step]
  macd_yesterday = macd[step - 1] - ema9[step]
  macd_today = macd[step] - ema9[step]
  signal = 'hold'
  if macd_yesterday > 0 and macd_today < 0:
    signal = 'buy'
  elif macd_yesterday < 0 and macd_today > 0:
    signal = 'sell'
  if signal == 'buy':
    affordable_qty = floor((cash - TRANS_COST) / today_price)
    if affordable_qty >= MIN_PURCHAS:
      cash -= today_price * affordable_qty
      stocks += affordable_qty
  if signal == 'sell':
    if stocks > 0:
      cash += today_price * stocks
      stocks = 0
  equity.append(cash)
  inventory.append(stocks)
  signals.append(signal)

today_date = iuit_l.index[step]
print('Date = {}, price = {:.2f}, signal = {}'.format(today_date,
                                                      today_price,
                                                      signal))

In [ ]:
plotPrices(prices, signals, ENTRY_IDX, None, None)

A trading strategy incorporating MACD and trend analysis involves immediate selling upon a trend reversal from uptrend to downtrend, provided the current price exceeds the last purchase price. In a rising market, standard MACD crossovers are utilized. This approach mitigates losses by avoiding conflicting buy and sell signals during market downturns. However, its reliance on a rolling mean trend indicator may lead to missed opportunities in situations where the price is declining despite a positive trend indication.

In [ ]:
trend = [0 if isnan(x) else x for x in pd.Series(prices).rolling(100).mean().tolist()]
trend_diffs = [0] + np.diff(trend).tolist()
trend_direction = ['up' if x > 0 else 'down' if x < 0 else 'neutral' for x in trend_diffs]

for x in range(1, len(trend_direction) - 1):
  yesterday = trend_direction[x - 1]
  today     = trend_direction[x]
  tomorrow  = trend_direction[x + 1]
  if yesterday == tomorrow:
    today = yesterday
  trend_direction[x] = today

trend = trend[ENTRY_IDX:]
trend_direction = trend_direction[ENTRY_IDX:]
tmp = pd.DataFrame({'price':trend, 'trend':trend_direction})

plt.plot(trend, color='lightgrey')
plt.plot(np.where(tmp['trend'] == 'down', tmp['price'], None), color='red')
plt.plot(np.where(tmp['trend'] == 'up', tmp['price'], None), color='green')
plt.show()

In [ ]:
trend = [0 if isnan(x) else x for x in pd.Series(prices).rolling(100).mean().tolist()]
trend_diffs = [0] + np.diff(trend).tolist()
trend_direction = ['up' if x > 0 else 'down' if x < 0 else 'neutral' for x in trend_diffs]

for x in range(1, len(trend_direction) - 1):
  yesterday = trend_direction[x - 1]
  today     = trend_direction[x]
  tomorrow  = trend_direction[x + 1]
  if yesterday == tomorrow:
    today = yesterday
  trend_direction[x] = today

ema12 = pd.Series(prices).ewm(12).mean()
ema26 = pd.Series(prices).ewm(26).mean()
macd  = ema12 - ema26
ema9  = macd.ewm(9).mean()
macd  = macd.tolist()
ema9  = ema9.tolist()

equity    = []
inventory = []
signals   = []

cash   = INITIAL_INV
stocks = 0

last_sales_price = float('inf')
last_buy_price = 0.0

steps = 0

for step in range(ENTRY_IDX, len(returns)):
  today_trend = trend_direction[step]
  yday_trend  = trend_direction[step - 1]
  today_price = prices[step]
  macd_yesterday = macd[step - 1] - ema9[step]
  macd_today = macd[step] - ema9[step]
  signal = 'hold'
  if macd_yesterday > 0 and macd_today < 0:
    signal = 'buy'
  elif macd_yesterday < 0 and macd_today > 0:
    signal = 'sell'

  if today_trend == 'down' and signal == 'buy':
    signal = 'hold'

  if today_trend == 'down' and yday_trend != 'down':
    signal = 'sell'

  if today_trend == 'down' and signals[-1] == 'buy':
    signal = 'sell'

  if signal == 'buy':
    if today_price >= last_sales_price:
      signal = 'hold'

  if signal == 'sell':
    if today_price <= last_buy_price:
      signal = 'hold'

  if signal == 'buy':
    affordable_qty = floor((cash - TRANS_COST) / today_price)
    if affordable_qty >= MIN_PURCHAS:
      cash -= today_price * affordable_qty
      stocks += affordable_qty
      last_buy_price = today_price
  if signal == 'sell':
    if stocks > 0:
      cash += today_price * stocks
      stocks = 0
      last_sales_price = today_price

  equity.append(cash)
  inventory.append(stocks)
  signals.append(signal)
  steps += 1

profit = 100 * (equity[-1] - INITIAL_INV) / INITIAL_INV
print('Profit: MACD crossover with trend = {:.2f}%'.format(profit))

d_now = iuit_l.index[step]
print('Date = {}, price = {:.4f}, signal = {}'.format(d_now, today_price, signal))

In [ ]:
plotPrices(prices, signals, ENTRY_IDX, None, None)

## ARMA-GARCH

In [ ]:
def computeAcfValue(diffs, n_lags):
  acf_res = pd.DataFrame(acf(pd.Series(diffs), nlags=n_lags))
  acf_res.columns = ['value']
  # ACF confidence interval is not equal to the marked region of the ACF plot
  # since the region is the critical region and it is computed differently from
  # the confidence interval. Source:
  # https://stats.stackexchange.com/questions/185425/how-to-determine-the-critical-values-of-acf
  # formula: 1.96 / sqrt(T - d), where T -- number of observations, d - lag
  critical_region = []
  for lag in range(1, n_lags + 2):
    critical_region.append(np.round(1.96 / np.sqrt(len(diffs) - lag), 5))
  acf_res['region'] = critical_region
  acf_res['abs_val'] = abs(acf_res['value'])
  acf_res = acf_res[acf_res.index != 0]
  signif_idx = acf_res.index[acf_res['abs_val'] >= acf_res['region']].tolist()
  if len(signif_idx) > 0:
    idx_list = [1 if x in signif_idx else 0 for x in range(1, max(signif_idx) + 1)]
  else:
    idx_list = [0]
  idx_list   = tuple(idx_list)
  return idx_list

def computePacfValue(diffs, n_lags):
  pacf_res = pd.DataFrame(pacf(pd.Series(diffs), nlags=n_lags))
  pacf_res.columns = ['value']
  critical_region  = []
  for lag in range(1, n_lags + 2):
    critical_region.append(np.round(1.96 / np.sqrt(len(diffs) - lag), 5))
  pacf_res['region']  = critical_region
  pacf_res['abs_val'] = abs(pacf_res['value'])
  pacf_res = pacf_res[pacf_res.index != 0]
  signif_idx = pacf_res.index[pacf_res['abs_val'] >= pacf_res['region']].tolist()
  if len(signif_idx) > 0:
    idx_list = [1 if x in signif_idx else 0 for x in range(1, max(signif_idx) + 1)]
  else:
    idx_list = [0]
  idx_list   = tuple(idx_list)
  return idx_list

def findAcfValue(acf_val):
  """
  This function takes a sequence of ACF lags starting from 1 and up to n-lags to
  return the number of lags as a list of indices that are equal to 1. In case
  the length of the ACF is 1, then it just returns the first value.
  """
  new_acf = 0
  if len(acf_val) == 1:
    new_acf = acf_val[0]
  else:
    new_acf = [i + 1 for i,x in enumerate(acf_val) if x == 1]
  if new_acf == 0:
    new_acf = [new_acf]
  return new_acf


def prepareOrders(max_p, max_q, ar, ma):
  """
  This function generates a list of candidate orders for ARMA model by combining
  pre-computed ACF and PACF values with default orders and removing (0, 0) to
  prevent random walk.
  ar -- pre-computed ACF value
  ma -- pre-computed PACF value
  Returns a list of tuples where the first value in a tuple is for P, the second
  is for Q.
  """
  orders = list(product(np.append(np.arange(max_p + 1), ar),
                        np.append(np.arange(max_q + 1), ma)))
  orders.remove((0, 0))
  return orders

def simultaneousLlh(r, orders):
  """
  This function employs the SLSQP optimization algorithm to simultaneously
  determine the optimal orders of ARMA and GARCH models, evaluating log
  likelihood using epsilon from ARMA and sigma from GARCH, and returning the
  P and Q values minimizing AIC.
  """
  best_aic = np.inf
  for order in orders:
    p = order[0]
    q = order[1]
    result = trainModel(r, p, q)
    current_aic = 2 * result[1] + 2 * len(result[0])
    if current_aic < best_aic:
      best_aic = current_aic
      best_p, best_q = p, q
      best_params = result[0]
  return best_p, best_q

def trainArma(r, p, q, max_p, max_q):
  """
  This function trains an ARMA model with specified P and Q parameters. If
  either parameter exceeds the predetermined threshold, the model transforms
  them into a list to represent standalone t - n lags, as described in
  the SARIMAX model.
  """
  if p > max_p:
    p = [p]
  if q > max_q:
    q = [q]
  arima_model = sm.tsa.statespace.SARIMAX(r,
                                          trend = 'c',
                                          order = (p, 0, q),
                                          enforce_stationarity=False,
                                          enforce_invertibility=False)
  arima_result = arima_model.fit(disp=False)
  arima_resids = arima_result.resid
  return arima_result, arima_resids

def computeEpsilon(c, phi, theta, r):
  """
  This function computes epsilon for using the formula for an ARMA(p, q) process
  with intercept and returns an array of epsilons.
  c - coefficient of intercept
  phi - list of phi coefficients of length p
  theta - list of theta coefficients of length q
  r - an array of returns
  """
  T = len(r)
  eps = np.zeros(T)
  for t in range(T):
    if t < len(phi):
      eps[t] = r[t] - np.mean(r)
    else:
      ar_part = np.sum(np.array([phi[i] * r[t - 1 - i] for i in range(len(phi))], dtype=np.float64))
      ma_part = np.sum(np.array([theta[i] * eps[t - 1- i] for i in range(len(theta))], dtype=np.float64))
      eps[t]  = r[t] - c - ar_part - ma_part
  return eps

def getSigma2(omega, alpha, beta, gamma, r, eps):
  T = len(eps)
  sigma2 = np.zeros(T)
  if (1 - alpha - beta) == 0:
    sigma2[0] = omega / (1 - 0.9999999999999998)
  else:
    sigma2[0] = omega / (1 - alpha - beta)
  for t in range(1, T):
    sigma2[t] = omega + alpha * eps[t - 1] ** 2 + beta * sigma2[t - 1]
  return sigma2

def llhNormal(params, p, q, r):
  # We need constant, phi and theta for epsilon estimation in ARMA process
  c     = params[0]
  phi   = params[1:p+1]
  theta = params[p+1:p+q+1]
  # We need omega, alpha and beta params for GARCH process
  omega, alpha, beta = params[-3:]
  gamma = None
  et = computeEpsilon(c, phi, theta, r)
  sigma2 = getSigma2(omega, alpha, beta, gamma, r, et)
  if any(sigma2 < 0):
    sigma2[sigma2 < 0] = -1 * sigma2[sigma2 < 0]
  llh    = -0.5 * (np.log(2*np.pi) + np.log(sigma2) + et**2 / (2*sigma2))
  neg_llh   = -llh
  total_llh = np.sum(neg_llh)
  return total_llh

def cons0(params, p, q, r):
  alpha, beta = params[-2:]
  return 1.0 - np.finfo(np.float64).eps - alpha - beta

def cons1(params, p, q, r):
  return 1.0 - np.sum(params[1:p+1]) - np.finfo(np.float64).eps

def trainModel(r, p, q):
  np.seterr(divide='ignore', invalid='ignore', over='ignore')
  e = np.finfo(np.float64).eps
  bounds = [(-10*np.abs(np.mean(r)), 10*np.abs(np.mean(r)))] + \
           [(-0.999999, 0.999999) for _ in range(p + q)] + \
           [(e, 2 * np.var(r))]
  alpha_bounds, beta_bounds = [(e, 1.0 - e) for _ in range(2)]
  initial_params = [0.001 for _ in range(p + q + 1)]
  initial_params = initial_params + [0.001, 0.1, 0.8]
  bounds = bounds + [alpha_bounds, beta_bounds]
  min_func = llhNormal
  eqcons   = []
  ieqcons  = [cons0, cons1]
  result = optimize.fmin_slsqp(func = min_func,
                               x0   = initial_params,
                               ieqcons = ieqcons,
                               eqcons  = eqcons,
                               bounds  = bounds,
                               epsilon = 1e-6,
                               acc     = 1e-7,
                               full_output = True,
                               iprint  = 0,
                               args    = (p, q, r),
                               iter    = 300)
  return result

def trainGarch(scaled_training):
  archm = arch_model(scaled_training,
                     mean = 'zero',
                     vol  = 'garch',
                     p    = 1,
                     q    = 1,
                     dist = 'Normal')
  gjr = archm.fit(update_freq=100, disp=False)
  return gjr

def findLengthOfSequences(nums):
  """
  This function finds positions of 'negative' strings in a list and returns them
  together with the sequence of 'positive' strings preceeding the negative ones.
  The purpose of this function is to produce an idea of how biased the random
  process is.
  """
  indexes = []
  result  = []
  count  = 0
  for idx in range(len(nums)):
    num = nums[idx]
    if num == 'positive':
      count += 1
    else:
      if count > 0:
        indexes.append(idx)
        result.append(count)
      count = 0
  if count > 0:
    indexes.append(idx)
    result.append(count)
  return {'indexes': indexes, 'counts':result}

In [ ]:
# Test how long can the period of positive return can be before it is replaced
# with negative to determine the sales point
seqs = ['positive' if x >= 0 else 'negative' for x in returns[:ENTRY_IDX]]
sign_seqs = pd.DataFrame(findLengthOfSequences(seqs))

pos = seqs.count('positive')
neg = seqs.count('negative')
print(pos, neg)

In [ ]:
temp = sign_seqs.groupby('counts').size().rename('frequency').reset_index()

In [ ]:
steps = 0


# TODO: change to upper and lower history

test_h = []
up_h   = []
down_h = []

for step in range(ENTRY_IDX, last_idx):
  window_rets = returns[step - LOOK_BACK:step + LOOK_AHEAD]
  train = window_rets[:LOOK_BACK]
  test  = window_rets[-LOOK_AHEAD:]

  p_max = max(findAcfValue(computeAcfValue(train, 20)))
  q_max = max(findAcfValue(computePacfValue(train, 20)))

  candidates = prepareOrders(MAX_P, MAX_Q, p_max, q_max)

  p, q = simultaneousLlh(train, candidates)
  arima_result, arima_resids = trainArma(train, p, q, MAX_P, MAX_Q)

  scaler = StandardScaler()
  scaled_train = scaler.fit_transform(np.array(arima_resids).reshape(-1, 1))
  gjr = trainGarch(scaled_train)

  predicted_mu = arima_result.forecast(steps=LOOK_AHEAD).tolist()

  predicted_et = gjr.forecast(horizon=len(test)).variance
  predicted_et = scaler.inverse_transform(predicted_et).tolist()[0]


  y_hat = [mu + et for mu, et in zip(predicted_mu, predicted_et)]

  steps += 1
  if steps == 1:
    break

In [ ]:
plt.plot(tmp['test'], color='grey')
plt.plot(tmp['pred'], color='green')
plt.show()